In [1]:
# Установка необходимых пакетов (выполнить один раз)
!pip install chromadb sentence-transformers pypdf2 requests pdfminer pdfminer.six

In [2]:
# Установка библтотеки для реализации простого UI
!pip install -U ipywidgets==7.7.1   

In [3]:
# Импорт библиотек
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets
from IPython.display import clear_output
import requests
import json
import re
import io
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text as pdfminer_extract
import multiprocessing
from typing import Optional
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [4]:
# Импорт wandb и метрик оценки
import wandb
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [5]:
# Авторизация в wandb (или настройка через .netrc)
wandb.login(key="d8feca2d41600ef144d9b8c9fc65e09625d2fa8c")
wandb.init(project="rag-eval", name="ChromaDB + LLM response evaluation")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\H4rdwork\_netrc
wandb: Currently logged in as: yugik (yugik-tomsk-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
# Функция оценки BLEU и ROUGE
def evaluate_metrics(reference, hypothesis):
    # BLEU
    bleu_score = sentence_bleu([reference.split()], hypothesis.split())

    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)

    rouge1 = scores['rouge1'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return bleu_score, rouge1, rougeL

In [7]:
# Конфигурация
EMBEDDING_MODEL_NAME = "cointegrated/rubert-tiny2"
CHROMA_DB_PATH = "./chroma_db"
DOCUMENTS_DIR = "./documents"
LLM_API_URL = "http://localhost:1234/v1/chat/completions"  # URL вашего локального LLM API
LLM_MODEL =  "yandexgpt-5-lite-8b-instruct" # LLM модель которую будем использовать для генерации ответа

# Инициализация модели для эмбеддингов
print("Загрузка модели для эмбеддингов...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

# Инициализация ChromaDB
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH, settings=Settings(allow_reset=True))

Загрузка модели для эмбеддингов...


In [8]:
# Создание или получение коллекции
collection = chroma_client.get_or_create_collection(
    name="documents_collection",
    metadata={"hnsw:space": "cosine"}  # Используем косинусное расстояние
)

In [9]:
# Функция извлечения текста и з PDF файла.
def extract_text_from_pdf(pdf_path: str, 
                         engine: str = "pypdf2", 
                         parallel: bool = False) -> Optional[str]:
    """
    Извлекает текст из PDF файла с выбором движка и возможностью параллельной обработки
    
    Параметры:
        pdf_path: путь к PDF файлу
        engine: движок для извлечения ("pypdf2" или "pdfminer")
        parallel: использовать многопоточность (только для pypdf2)
    
    Возвращает:
        Текст документа или None при ошибке
    """
    def _extract_with_pypdf2(file_stream):
        """Внутренняя функция для pypdf2"""
        try:
            reader = PdfReader(file_stream)
            if parallel:
                with multiprocessing.Pool() as pool:
                    texts = pool.map(lambda page: page.extract_text(), reader.pages)
                return "\n".join(filter(None, texts))
            return "\n".join(page.extract_text() or "" for page in reader.pages)
        except Exception as e:
            print(f"Ошибка PyPDF2: {e}")
            return None

    def _extract_with_pdfminer(file_stream):
        """Внутренняя функция для pdfminer"""
        try:
            return pdfminer_extract(file_stream)
        except Exception as e:
            print(f"Ошибка pdfminer: {e}")
            return None

    try:
        with open(pdf_path, "rb") as f:
            file_stream = io.BytesIO(f.read())
            
            if engine == "pdfminer":
                return _extract_with_pdfminer(file_stream)
            else:
                return _extract_with_pypdf2(file_stream)
                
    except Exception as e:
        print(f"Ошибка чтения файла {pdf_path}: {e}")
        return None

In [10]:
# Функция разбивки текста на чанки
def chunk_text(text, chunk_size=450, overlap=50):
    """Разбивает текст на чанки с перекрытием"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [11]:
# Функция обработки документа или документов если их несколько
def process_documents():
    """Оптимизированная обработка PDF с параллельным выполнением и батчингом"""
    # Создаем директорию если не существует
    os.makedirs(DOCUMENTS_DIR, exist_ok=True)
    
    # Получаем список PDF файлов
    pdf_files = [f for f in os.listdir(DOCUMENTS_DIR) 
                if f.lower().endswith(".pdf")]
    if not pdf_files:
        print(f"В директории {DOCUMENTS_DIR} не найдено PDF файлов.")
        return

    print(f"Начата обработка {len(pdf_files)} PDF файлов...")

    def process_single_file(pdf_file):
        """Обработка одного PDF файла"""
        pdf_path = os.path.join(DOCUMENTS_DIR, pdf_file)
        try:
            text = extract_text_from_pdf(pdf_path)
            chunks = chunk_text(text)
            return [
                {
                    "text": chunk,
                    "metadata": {"source": pdf_file, "chunk_num": i},
                    "id": f"{pdf_file}_chunk_{i}"
                }
                for i, chunk in enumerate(chunks)
            ]
        except Exception as e:
            print(f"Ошибка обработки файла {pdf_file}: {e}")
            return []

    # Параллельная обработка файлов
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        results = list(tqdm(
            executor.map(process_single_file, pdf_files),
            total=len(pdf_files),
            desc="Обработка PDF файлов"
        ))

    # Подготовка данных для ChromaDB
    documents = []
    metadatas = []
    ids = []
    
    for result in results:
        for item in result:
            documents.append(item["text"])
            metadatas.append(item["metadata"])
            ids.append(item["id"])

    if not documents:
        print("Нет документов для добавления.")
        return

    # Оптимизированная генерация эмбеддингов с батчингом
    print("Генерация эмбеддингов...")
    batch_size = 128  # Оптимальный размер батча для большинства GPU
    embeddings = []
    
    for i in tqdm(range(0, len(documents), batch_size),
                    desc="Генерация эмбеддингов",
                    unit="batch"):
        batch = documents[i:i + batch_size]
        embeddings_batch = embedding_model.encode(
            batch,
            normalize_embeddings=True,
            convert_to_numpy=True,
            show_progress_bar=False
        )
        embeddings.append(embeddings_batch)
    
    embeddings = np.vstack(embeddings)

    # Пакетное добавление в ChromaDB (актуальный API)
    print("Добавление документов в ChromaDB...")
    collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas,
        embeddings=embeddings.tolist() if isinstance(embeddings, np.ndarray) else embeddings
    )
    
    print(f"Успешно добавлено {len(documents)} чанков из {len(pdf_files)} файлов")

    # Оптимизация индекса (для ChromaDB >= 0.4.0)
    if hasattr(collection, 'update_index'):
        collection.update_index()
        print("Индекс ChromaDB обновлен")
    else:
        print("Автоматическая оптимизация индекса не требуется")

In [ ]:
# Функция отправкм в LLM контекста и сообщения.
def query_llm_api(prompt, context, max_tokens=4096):
    messages = [
        {
            "role": "system",
            "content": """Ты - помощник, который отвечает на вопросы на основе предоставленного контекста. 
            Если ответа нет в контексте, скажи, что не знаешь. Не придумывай информацию."""
        },
        {
            "role": "user",
            "content": f"""Контекст: {context}\n\nВопрос: {prompt}"""
        }
    ]

    data={
     "model": LLM_MODEL,  
     "messages":  messages,
     "max_tokens": max_tokens,
     "temperature": 0.45
            }
    
    headers = {
    "Content-Type": "application/json" }
    
    response = requests.post(LLM_API_URL, headers=headers, data=json.dumps(data), timeout=180)
                  
    if response.status_code == 200:
        result = response.json()
        print(result["choices"][0]["message"]["content"])
    else:
        print(f"Error: {response.status_code}")

In [13]:
def ask_question(question):
    """Функция для обработки вопроса пользователя"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode(question, normalize_embeddings=True).tolist()

    # Ищем релевантные документы
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3  # Количество возвращаемых результатов
    )

    # Извлекаем контекст
    context = "\n\n".join(results["documents"][0])

    # Генерируем ответ с помощью LLM
    answer = query_llm_api(question, context)

    # Также получаем источники для ссылок
    sources = list(set([metadata["source"] for metadata in results["metadatas"][0]]))

    # Выводим результат
    display(Markdown(f"**Вопрос:** {question}"))
    #display(Markdown(f"**Ответ:** {answer}"))
    if sources:
        display(Markdown("**Источники:**"))
        for source in sources:
            display(Markdown(f"- {source}"))

In [14]:
# Функция очистки CromaDB от сохраненного в нее контекста.
def reset_database():
    """Сброс базы данных"""
    try:
        chroma_client.reset()
        global collection
        collection = chroma_client.get_or_create_collection(
            name="documents_collection",
            metadata={"hnsw:space": "cosine"}
        )
        print("База данных успешно сброшена.")
    except Exception as e:
        print(f"Ошибка при сбросе базы данных: {e}")

In [15]:
# Создаем интерфейс для взаимодействия
def create_interface():
    # Обработка документов при запуске
    process_documents()

    # Создаем элементы интерфейса
    question_input = widgets.Textarea(
        value='',
        placeholder='Введите ваш вопрос...',
        description='Вопрос:',
        layout={'width': '80%'},
        rows=3
    )

    ask_button = widgets.Button(description="Задать вопрос")
    reset_button = widgets.Button(description="Сбросить базу данных")
    output = widgets.Output()

    def on_ask_button_clicked(b):
        with output:
            clear_output()
            ask_question(question_input.value)

    def on_reset_button_clicked(b):
        with output:
            clear_output()
            reset_database()
            print("База данных сброшена. Загрузите документы заново.")

    ask_button.on_click(on_ask_button_clicked)
    reset_button.on_click(on_reset_button_clicked)

    # Отображаем интерфейс
    display(widgets.VBox([
        widgets.HBox([question_input, ask_button]),
        reset_button,
        output
    ]))

In [16]:
# Запускаем интерфейс для работы с LLM.
create_interface()

Начата обработка 1 PDF файлов...


Обработка PDF файлов: 100%|██████████| 1/1 [00:13<00:00, 13.82s/it]


Генерация эмбеддингов...


Генерация эмбеддингов: 100%|██████████| 1/1 [00:08<00:00,  8.19s/batch]


Добавление документов в ChromaDB...
Успешно добавлено 96 чанков из 1 файлов
Автоматическая оптимизация индекса не требуется
